<a href="https://colab.research.google.com/github/andrillion/travel-site/blob/master/Kopia_av_aco_osrm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ACO for the Traveling Salesman Problem

0. Install packages
1. Creating the ACO algorithm
2. Plot optimal tour on map with Folium
3. Plot progress over iterations

## 0. Install packages

In [ ]:
import sys
print(sys.executable)


/anaconda3/bin/python


In [ ]:
!{sys.executable} -m pip install geopy


In [ ]:
!{sys.executable} -m pip install requests

In [ ]:
!{sys.executable} -m pip install polyline

In [ ]:
!{sys.executable} -m pip install folium

## 1. Creating the ACO algorithm

In [ ]:
import numpy as np
import random
import requests
import polyline
from geopy.distance import great_circle

class AntColonyOptimizer:
    def __init__(self, num_cities, num_ants, evaporation_rate, alpha, beta, initial_pheromone, max_iterations):
        self.num_cities = num_cities
        self.num_ants = num_ants
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha  # Pheromone importance
        self.beta = beta    # Distance importance
        self.pheromone_matrix = np.full((num_cities, num_cities), initial_pheromone)
        self.distance_matrix = None  # To be initialized separately
        self.max_iterations = max_iterations

    def initialize_distance_matrix(self, distances):
        self.distance_matrix = distances

    def run(self):
        best_tour = None
        best_length = float('inf')
        length_history = []  # Initialize length history

        for iteration in range(self.max_iterations):
            print(f"Starting iteration {iteration + 1}/{self.max_iterations}")

            all_tours = self.construct_solution()
            self.update_pheromones(all_tours)

            for tour in all_tours:
                length = self.calculate_tour_length(tour)
                if length < best_length:
                    best_length = length
                    best_tour = tour

            length_history.append(best_length)  # Record the best length of this iteration

        return best_tour, best_length, length_history

    def construct_solution(self):
        all_tours = []
        for _ in range(self.num_ants):
            tour = self.build_ant_tour()
            all_tours.append(tour)
        return all_tours

    def build_ant_tour(self):
        tour = []
        visited = set()
        current_city = random.randint(0, self.num_cities - 1)
        tour.append(current_city)
        visited.add(current_city)

        for _ in range(self.num_cities - 1):
            next_city = self.select_next_city(current_city, visited)
            tour.append(next_city)
            visited.add(next_city)
            current_city = next_city

        return tour

    def select_next_city(self, current_city, visited):
        probabilities = []
        for city in range(self.num_cities):
            if city not in visited:
                trail = self.pheromone_matrix[current_city][city] ** self.alpha
                visibility = (1.0 / (self.distance_matrix[current_city][city] + 1e-8)) ** self.beta
                probabilities.append(trail * visibility)
            else:
                probabilities.append(0)

        probabilities = np.array(probabilities) / np.sum(probabilities)
        next_city = np.random.choice(self.num_cities, p=probabilities)
        return next_city

    def calculate_tour_length(self, tour):
        length = 0
        for i in range(len(tour) - 1):
            length += self.distance_matrix[tour[i]][tour[i+1]]
        length += self.distance_matrix[tour[-1]][tour[0]]  # Return to start
        return length

    def update_pheromones(self, all_tours):
        self.evaporate_pheromones()
        self.deposit_pheromones(all_tours)

    def evaporate_pheromones(self):
        self.pheromone_matrix *= (1 - self.evaporation_rate)

    def deposit_pheromones(self, all_tours):
        for tour in all_tours:
            contribution = self.calculate_tour_contribution(tour)
            for i in range(len(tour) - 1):
                self.pheromone_matrix[tour[i]][tour[i+1]] += contribution

    def calculate_tour_contribution(self, tour):
        return 1.0 / self.calculate_tour_length(tour)

def generate_city_distances_and_routes(locations):
    num_cities = len(locations)
    distance_matrix = np.zeros((num_cities, num_cities))
    route_geometries = {}  # Dictionary to store route geometries
    loc_list = list(locations.values())

    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                coords_i = loc_list[i]
                coords_j = loc_list[j]

                print(f"Processing distance between {list(locations.keys())[i]} and {list(locations.keys())[j]}")


                response = requests.get(f"http://router.project-osrm.org/route/v1/driving/{coords_i[1]},{coords_i[0]};{coords_j[1]},{coords_j[0]}?overview=full")
                data = response.json()
                distance = data['routes'][0]['distance'] / 1000  # Convert meters to kilometers
                distance_matrix[i][j] = distance
                # Extract and store the route geometry
                route_geometries[(i, j)] = data['routes'][0]['geometry']
                # After obtaining route_geometries from the function
                print(next(iter(route_geometries.values())))  # Print the first route geometry as

            else:
                distance_matrix[i][j] = float('inf')

    return distance_matrix, route_geometries

# Define locations in Stockholm
stockholm_locations = {
    "Gamla Stan": (59.3251, 18.0711),
    "Vasa Museum": (59.3294, 18.0915),
    "Skansen": (59.3257, 18.0961),
    "Royal Palace": (59.3269, 18.0737),
    "Djurgarden": (59.3298, 18.1167),
    "Fotografiska": (59.3177, 18.0844),
    "Gröna Lund": (59.3230, 18.0963),
    "Moderna Museet": (59.3263, 18.0846),
    "Stockholm City Hall": (59.3275, 18.0544),
    "ABBA The Museum": (59.3240, 18.0962),
    "Nobel Prize Museum": (59.3247, 18.0707),
    "Stockholm Cathedral": (59.3258, 18.0709),
    "Nordiska Museet": (59.3295, 18.0908),
    "Stockholm Olympic Stadium": (59.3459, 18.0794),
    "Kungstradgarden Park": (59.3318, 18.0702),
    "Monteliusvagen": (59.3190, 18.0581),
    "Ostermalm Market Hall": (59.3350, 18.0776),
    "The Royal Dramatic Theatre": (59.3308, 18.0735),
    "Humlegarden": (59.3382, 18.0728),
    "Stortorget": (59.3248, 18.0704)
}


# Script execution
num_cities = len(stockholm_locations)
distance_matrix, route_geometries = generate_city_distances_and_routes(stockholm_locations)

aco = AntColonyOptimizer(
    num_cities=num_cities,
    num_ants=5,
    evaporation_rate=0.1,
    alpha=1.0,
    beta=2.0,
    initial_pheromone=0.1,
    max_iterations=50
)
aco.initialize_distance_matrix(distance_matrix)

best_tour, best_length, length_history = aco.run()
print("Best Tour:", [list(stockholm_locations.keys())[i] for i in best_tour])
print("Best Length:", best_length)


Processing distance between Gamla Stan and Vasa Museum
s}aiJiphmBe@Bq@P[[B]{@qF]e@w@oFBi@AICICKCKAICKEQSXML}E`G[`@M@G?G?wFw@MEY??c@AWEiAGmAEk@IW_CgEyBsDEEICK?IJKZITQ\SRA@KDODM?KAA?SM[UIIKOSg@AEK[?KCQEYEe@?C?M?Y@WD[D[VwAv@oEBKF[tAwHFi@H{@@{@C}DAkB@s@?}@?kBC}@G{E]{ZE_FAwAEmEAK?MAeALQFGd@YdBw@d@Ox@MBVBr@Jx@DHHTZZj@\FD|@T^LZP
Processing distance between Gamla Stan and Skansen
s}aiJiphmBe@Bq@P[[B]{@qF]e@w@oFBi@AICICKCKAICKEQSXML}E`G[`@M@G?G?wFw@MEY??c@AWEiAGmAEk@IW_CgEyBsDEEICK?IJKZITQ\SRA@KDODM?KAA?SM[UIIKOSg@AEK[?KCQEYEe@?C?M?Y@WD[D[VwAv@oEBKF[tAwHFi@H{@@{@C}DAkB@s@?}@?kBC}@G{E]{ZE_FAwAEmEAK?MAeALQFGd@YdBw@d@Ox@MBVBr@Jx@DHHTZZj@\FD|@T^LZP
Processing distance between Gamla Stan and Royal Palace
s}aiJiphmBe@Bq@P[[B]{@qF]e@w@oFBi@AICICKCKAICKEQSXML}E`G[`@M@G?G?wFw@MEY??c@AWEiAGmAEk@IW_CgEyBsDEEICK?IJKZITQ\SRA@KDODM?KAA?SM[UIIKOSg@AEK[?KCQEYEe@?C?M?Y@WD[D[VwAv@oEBKF[tAwHFi@H{@@{@C}DAkB@s@?}@?kBC}@G{E]{ZE_FAwAEmEAK?MAeALQFGd@YdBw@d@Ox@MBVBr@Jx@DHHTZZj@\FD|@T^LZP
Processing distance between Gam

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import folium
import polyline

def plot_tour_on_map(tour, locations, route_geometries):
    if not tour:
        print("No tour data available.")
        return None

    starting_location = list(locations.keys())[tour[0]]
    map = folium.Map(location=locations[starting_location], zoom_start=12)

    # Add markers for each location
    for loc_name, coords in locations.items():
        folium.Marker(coords, popup=loc_name).add_to(map)

    # Plot each leg of the journey using the route geometries
    for i in range(len(tour) - 1):
        start = tour[i]
        end = tour[i + 1]

        # Diagnostics: Print start and end locations
        print(f"Plotting leg from {list(locations.keys())[start]} to {list(locations.keys())[end]}")

        # Check if the route geometry for this leg exists
        if (start, end) in route_geometries:
            # Decode the polyline and plot the route
            route_coords = polyline.decode(route_geometries[(start, end)], geojson=True)
            folium.PolyLine(route_coords, color="blue", weight=2.5, opacity=1).add_to(map)
        else:
            print(f"No route geometry found for leg from {list(locations.keys())[start]} to {list(locations.keys())[end]}")

    return map

# Assuming 'best_tour', 'stockholm_locations', and 'route_geometries' are available from Part 1
tour_map = plot_tour_on_map(best_tour, stockholm_locations, route_geometries)

# Save the map to an HTML file and display it
if tour_map:
    tour_map.save("tsp_solution_2024.html")
    display(tour_map)  # Make sure to import display from IPython.display
else:
    print("Failed to create the tour map.")


In [ ]:
# Print first few coordinates of the decoded route
print(decoded_route[:5])


In [ ]:
# Sample encoded polyline
encoded_polyline = route_geometries.get((best_tour[0], best_tour[1]))

# Decode and print the polyline
decoded_coords = polyline.decode(encoded_polyline, geojson=True)
print(decoded_coords[:5])  # Print first few coordinates


In [ ]:
# Check the first few entries in route_geometries
for key in list(route_geometries.keys())[:5]:
    print(key, polyline.decode(route_geometries[key], geojson=True)[:5])


In [ ]:
for i in range(len(best_tour) - 1):
    start = best_tour[i]
    end = best_tour[i + 1]

    if (start, end) in route_geometries:
        route_coords = polyline.decode(route_geometries[(start, end)], geojson=True)
        folium.PolyLine(route_coords, color="blue", weight=2.5, opacity=1).add_to(map)
    else:
        print(f"Route geometry not found for leg: {start} to {end}")


In [ ]:
display(map)


In [ ]:
import folium
import polyline

def plot_tour_on_map(tour, locations, route_geometries):
    # Check if the tour data is available
    if not tour:
        print("No tour data available.")
        return None

    # Starting location for the tour
    starting_location = list(locations.keys())[tour[0]]
    map = folium.Map(location=locations[starting_location], zoom_start=12)

    # Adding markers for each location
    for loc_name, coords in locations.items():
        folium.Marker(coords, popup=loc_name).add_to(map)

    # Plotting each leg of the tour
    for i in range(len(tour)):
        start = tour[i]
        end = tour[(i + 1) % len(tour)]  # Ensure loop back to start

        # Decode the polyline and plot the route if geometry exists
        if (start, end) in route_geometries:
            encoded_polyline = route_geometries[(start, end)]
            decoded_coords = polyline.decode(encoded_polyline, geojson=True)
            folium.PolyLine(decoded_coords, color="blue", weight=2.5, opacity=10).add_to(map)
        else:
            print(f"Missing geometry for tour leg: {start} to {end}")

    return map

# Assuming 'best_tour', 'stockholm_locations', and 'route_geometries' are available from Part 1
tour_map = plot_tour_on_map(best_tour, stockholm_locations, route_geometries)

# Save the map to an HTML file and display it
if tour_map:
    tour_map.save("tsp_solution_2024.html")
    display(tour_map)  # Make sure to import display from IPython.display
else:
    print("Failed to create the tour map.")


In [ ]:
import numpy as np
import random
from geopy.distance import great_circle

class AntColonyOptimizer:
    def __init__(self, num_cities, num_ants, evaporation_rate, alpha, beta, initial_pheromone, max_iterations):
        self.num_cities = num_cities
        self.num_ants = num_ants
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha  # Pheromone importance
        self.beta = beta    # Distance importance
        self.pheromone_matrix = np.full((num_cities, num_cities), initial_pheromone)
        self.distance_matrix = None  # To be initialized separately
        self.max_iterations = max_iterations

    def initialize_distance_matrix(self, distances):
        self.distance_matrix = distances

    def run(self):
        best_tour = None
        best_length = float('inf')
        length_history = []  # Initialize length history



        for _ in range(self.max_iterations):
            all_tours = self.construct_solution()
            self.update_pheromones(all_tours)

            for tour in all_tours:
                length = self.calculate_tour_length(tour)
                if length < best_length:
                    best_length = length
                    best_tour = tour

            length_history.append(best_length)  # Record the best length of this iteration


        return best_tour, best_length, length_history

    def construct_solution(self):
        all_tours = []
        for _ in range(self.num_ants):
            tour = self.build_ant_tour()
            all_tours.append(tour)
        return all_tours

    def build_ant_tour(self):
        tour = []
        visited = set()
        current_city = random.randint(0, self.num_cities - 1)
        tour.append(current_city)
        visited.add(current_city)

        for _ in range(self.num_cities - 1):
            next_city = self.select_next_city(current_city, visited)
            tour.append(next_city)
            visited.add(next_city)
            current_city = next_city

        return tour

    def select_next_city(self, current_city, visited):
        probabilities = []
        for city in range(self.num_cities):
            if city not in visited:
                trail = self.pheromone_matrix[current_city][city] ** self.alpha
                visibility = (1.0 / self.distance_matrix[current_city][city]) ** self.beta
                probabilities.append(trail * visibility)
            else:
                probabilities.append(0)

        probabilities = np.array(probabilities) / np.sum(probabilities)
        next_city = np.random.choice(self.num_cities, p=probabilities)
        return next_city

    def calculate_tour_length(self, tour):
        length = 0
        for i in range(len(tour) - 1):
            length += self.distance_matrix[tour[i]][tour[i+1]]
        length += self.distance_matrix[tour[-1]][tour[0]]  # Return to start
        return length

    def update_pheromones(self, all_tours):
        self.evaporate_pheromones()
        self.deposit_pheromones(all_tours)

    def evaporate_pheromones(self):
        self.pheromone_matrix *= (1 - self.evaporation_rate)

    def deposit_pheromones(self, all_tours):
        for tour in all_tours:
            contribution = self.calculate_tour_contribution(tour)
            for i in range(len(tour) - 1):
                self.pheromone_matrix[tour[i]][tour[i+1]] += contribution

    def calculate_tour_contribution(self, tour):
        return 1.0 / self.calculate_tour_length(tour)

# Define 20 popular locations in Stockholm with their geographical coordinates
stockholm_locations = {
    "Gamla Stan": (59.3251, 18.0711),
    "Vasa Museum": (59.3294, 18.0915),
    "Skansen": (59.3257, 18.0961),
    "Royal Palace": (59.3269, 18.0737),
    "Djurgarden": (59.3298, 18.1167),
    "Fotografiska": (59.3177, 18.0844),
    "Gröna Lund": (59.3230, 18.0963),
    "Moderna Museet": (59.3263, 18.0846),
    "Stockholm City Hall": (59.3275, 18.0544),
    "ABBA The Museum": (59.3240, 18.0962),
    "Nobel Prize Museum": (59.3247, 18.0707),
    "Stockholm Cathedral": (59.3258, 18.0709),
    "Nordiska Museet": (59.3295, 18.0908),
    "Stockholm Olympic Stadium": (59.3459, 18.0794),
    "Kungstradgarden Park": (59.3318, 18.0702),
    "Monteliusvagen": (59.3190, 18.0581),
    "Ostermalm Market Hall": (59.3350, 18.0776),
    "The Royal Dramatic Theatre": (59.3308, 18.0735),
    "Humlegarden": (59.3382, 18.0728),
    "Stortorget": (59.3248, 18.0704)
}


def generate_city_distances_and_routes(locations):
    num_cities = len(locations)
    distance_matrix = np.zeros((num_cities, num_cities))
    route_geometries = {}  # Dictionary to store route geometries
    loc_list = list(locations.values())

    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                coords_i = loc_list[i]
                coords_j = loc_list[j]
                response = requests.get(f"http://router.project-osrm.org/route/v1/driving/{coords_i[1]},{coords_i[0]};{coords_j[1]},{coords_j[0]}?overview=full")
                data = response.json()
                distance = data['routes'][0]['distance'] / 1000  # Convert meters to kilometers
                distance_matrix[i][j] = distance
                # Extract and store the route geometry
                route_geometries[(i, j)] = data['routes'][0]['geometry']
            else:
                distance_matrix[i][j] = float('inf')

    return distance_matrix, route_geometries


# Script execution
num_cities = len(stockholm_locations)
aco = AntColonyOptimizer(
    num_cities=num_cities,
    num_ants=5,
    evaporation_rate=0.1,
    alpha=1.0,
    beta=2.0,
    initial_pheromone=0.1,
    max_iterations=100
)
aco.initialize_distance_matrix(generate_city_distances(stockholm_locations))

best_tour, best_length, length_history = aco.run()
print("Best Tour:", [list(stockholm_locations.keys())[i] for i in best_tour])
print("Best Length:", best_length)


## 2. Plot optimal tour on map with Folium

In [ ]:
import folium

def plot_tour_on_map(tour, locations, starting_location):
    # Create a map centered around the starting location
    map = folium.Map(location=locations[starting_location], zoom_start=12)

    # Add markers for each location
    for loc_name, coords in locations.items():
        folium.Marker(coords, popup=loc_name).add_to(map)

    # Draw lines for the tour
    tour_coords = [locations[list(locations.keys())[i]] for i in tour] + [locations[starting_location]]
    folium.PolyLine(tour_coords, color="blue", weight=2.5, opacity=1).add_to(map)

    return map

# Script execution
#num_cities = len(stockholm_locations)
#aco = AntColonyOptimizer(
#    num_cities=num_cities,
#    num_ants=5,
#    evaporation_rate=0.1,
#    alpha=1.0,
#    beta=2.0,
#    initial_pheromone=0.1,
#    max_iterations=100
#)
#aco.initialize_distance_matrix(generate_city_distances(stockholm_locations))

#best_tour, best_length = aco.run()
#print("Best Tour:", [list(stockholm_locations.keys())[i] for i in best_tour])
#print("Best Length:", best_length)

# Plot the tour on the map
starting_location = list(stockholm_locations.keys())[best_tour[0]]
tour_map = plot_tour_on_map(best_tour, stockholm_locations, starting_location)


# Save the map as an HTML file
tour_map.save("tsp_solution_2024.html")

# Display the map
tour_map


## 3. Plot progress over iterations

In [ ]:
import matplotlib.pyplot as plt


# Plot optimization progress
plt.figure(figsize=(10, 6))
plt.plot(length_history, marker='o')
plt.title('ACO Optimization Progress')
plt.xlabel('Iteration')
plt.ylabel('Total Distance of Best Tour')
plt.grid(True)
plt.show()

## Use OSRM to calculate real world road distances between locations

In [ ]:
import requests

def generate_city_distances_osrm(locations):
    num_cities = len(locations)
    distance_matrix = np.zeros((num_cities, num_cities))
    loc_list = list(locations.values())

    for i in range(num_cities):
        for j in range(i+1, num_cities):
            coords_i = loc_list[i]
            coords_j = loc_list[j]
            response = requests.get(f"http://router.project-osrm.org/route/v1/driving/{coords_i[1]},{coords_i[0]};{coords_j[1]},{coords_j[0]}?overview=false")
            data = response.json()
            distance = data['routes'][0]['distance'] / 1000  # Convert meters to kilometers
            distance_matrix[i][j] = distance_matrix[j][i] = distance

    return distance_matrix


In [ ]:
def generate_city_distances_and_routes(locations):
    num_cities = len(locations)
    distance_matrix = np.zeros((num_cities, num_cities))
    route_geometries = {}
    loc_list = list(locations.values())

    for i in range(num_cities):
        for j in range(i+1, num_cities):  # Adjust loop to skip redundant calculations
            coords_i = loc_list[i]
            coords_j = loc_list[j]
            response = requests.get(f"http://router.project-osrm.org/route/v1/driving/{coords_i[1]},{coords_i[0]};{coords_j[1]},{coords_j[0]}?overview=full")
            data = response.json()
            distance = data['routes'][0]['distance'] / 1000  # Convert meters to kilometers
            distance_matrix[i][j] = distance_matrix[j][i] = distance  # Assume bidirectionality
            route_geometries[(i, j)] = data['routes'][0]['geometry']
            route_geometries[(j, i)] = data['routes'][0]['geometry']  # Same route in reverse

    return distance_matrix, route_geometries


In [ ]:
import requests

def generate_city_distances_and_routes(locations):
    num_cities = len(locations)
    distance_matrix = np.zeros((num_cities, num_cities))
    route_geometries = {}
    loc_list = list(locations.values())

    for i in range(num_cities):
        for j in range(i + 1, num_cities):  # Optimized to avoid redundant calculations
            coords_i = loc_list[i]
            coords_j = loc_list[j]
            response = requests.get(f"http://router.project-osrm.org/route/v1/driving/{coords_i[1]},{coords_i[0]};{coords_j[1]},{coords_j[0]}?overview=full")

            # Check if the response is valid
            if response.status_code == 200:
                data = response.json()

                # Check if the response contains the expected data
                if data.get('routes'):
                    distance = data['routes'][0]['distance'] / 1000  # Convert meters to kilometers
                    distance_matrix[i][j] = distance_matrix[j][i] = distance
                    route_geometries[(i, j)] = route_geometries[(j, i)] = data['routes'][0]['geometry']
                else:
                    print(f"Warning: No route data found for locations {i} and {j}")
            else:
                print(f"Error: OSRM API call failed for locations {i} and {j} with status code {response.status_code}")

    return distance_matrix, route_geometries


In [ ]:
import folium

# Create a basic map
basic_map = folium.Map(location=[59.3293, 18.0686], zoom_start=12)  # Coordinates for Stockholm

# Add a marker
folium.Marker([59.3293, 18.0686], popup='Stockholm').add_to(basic_map)

# Save to HTML and try displaying in the notebook
basic_map.save("basic_map_test.html")
basic_map
